<a href="https://colab.research.google.com/github/LeeMinJun0102/itwill_middle_project/blob/main/%EB%B6%84%EC%84%9D(sklearn).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, r2_score, f1_score
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import scipy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# DataFrame rk 0, 1, 2 등급화

# 데이터 준비

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/중간 프로젝트/코랩으로 옮길 엑셀들/DF_final_final.xlsx')

## only you 홍천 데이터 준비

In [ ]:
gangwon = pd.read_excel('/content/drive/MyDrive/중간 프로젝트/코랩으로 옮길 엑셀들/for_hongchun.xlsx')

In [ ]:
hongchun = gangwon[gangwon['ADM_NM'] == '강원도홍천군']

In [ ]:
hongchun_grid = hongchun['grid_id'].values

In [ ]:
yes_hong_df = df[df['grid_id'].isin(hongchun_grid)]

## without you 홍천 데이터 준비

In [ ]:
no_hong_df = df[~df['grid_id'].isin(hongchun_grid)]

In [ ]:
no_hong_df

# X, y, X_hong, y_hong

In [ ]:
X = no_hong_df[['wildlife_corridor', 'road_count', 'speed_limit', 'white_road_grade', 'pig_count', 'rani_count', 'dist_to_eco_m']].values
# X = no_hong_df[['wildlife_corridor', 'road_count', 'speed_limit', 'white_road_grade', 'pig_count', 'rani_count']].values
y = no_hong_df['rk_012'].values

In [ ]:
X_hong = yes_hong_df[['wildlife_corridor', 'road_count', 'speed_limit', 'white_road_grade', 'pig_count', 'rani_count', 'dist_to_eco_m']].values
# X_hong = yes_hong_df[['wildlife_corridor', 'road_count', 'speed_limit', 'white_road_grade', 'pig_count', 'rani_count']].values
y_hong = yes_hong_df['rk_012'].values

In [ ]:
df['total_rk'].value_counts()

# Classifier

## KNN

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
knn_train_pred = knn.predict(X_train)
knn_test_pred = knn.predict(X_test)

In [ ]:
confusion_matrix(y_train, knn_train_pred)

array([[426,   0,   0],
       [  0,  82,   7],
       [  0,  14,  11]])

In [ ]:
print('train f1_score: ', f1_score(y_train, knn_train_pred, average=None))
print('test f1_score', f1_score(y_test, knn_test_pred, average=None))

In [ ]:
print('train acc: ', knn.score(X_train, y_train))
print('test acc: ', knn.score(X_test, y_test))

## Log

In [ ]:
log = LogisticRegression()

In [ ]:
log.fit(X_train, y_train)

In [ ]:
log_train_pred = log.predict(X_train)
log_test_pred = log.predict(X_test)

In [ ]:
print('train r2_score: ', r2_score(y_train, log_train_pred))
print('test r2_score: ', r2_score(y_test, log_test_pred))

In [ ]:
print('train acc: ', log.score(X_train, y_train))
print('test acc: ', log.score(X_test, y_test))

## Decision Tree Classifier

In [ ]:
tree = DecisionTreeClassifier(max_depth=5)

In [ ]:
tree.fit(X_train, y_train)

In [ ]:
tree_train_pred = tree.predict(X_train)
tree_test_pred = tree.predict(X_test)

In [ ]:
print('train r2_score: ', r2_score(y_train, tree_train_pred))
print('test r2_score: ', r2_score(y_test, tree_test_pred))

In [ ]:
print('train acc: ', tree.score(X_train, y_train))
print('test acc: ', tree.score(X_test, y_test))

## SVM

In [ ]:
svm = SVC(probability=True)

In [ ]:
svm.fit(X_train, y_train)

In [ ]:
svm_train_pred = svm.predict(X_train)
svm_test_pred = svm.predict(X_test)

In [ ]:
print('train r2_score: ', r2_score(y_train, svm_train_pred))
print('test r2_score: ', r2_score(y_test, svm_test_pred))

In [ ]:
print('train acc: ', svm.score(X_train, y_train))
print('test acc: ', svm.score(X_test, y_test))

## SGD

In [ ]:
sgd = SGDClassifier(loss='log_loss')

In [ ]:
sgd.fit(X_train, y_train)

In [ ]:
sgd_train_pred = sgd.predict(X_train)
sgd_test_pred = sgd.predict(X_test)

In [ ]:
print('train acc: ', sgd.score(X_train, y_train))
print('test acc: ', sgd.score(X_test, y_test))

# 앙상블

## XXX Voting

In [ ]:
vote = VotingClassifier(estimators=[('knn', knn),
                                    ('tree', tree),
                                    ('logistic', log),
                                    ('sgd', sgd),
                                    ('svm', svm)],
                        voting='soft',
                        n_jobs=-1)

In [ ]:
vote.fit(X_train, y_train)

In [ ]:
print(vote.score(X_train, y_train))
print(vote.score(X_test, y_test))

In [ ]:
cv_vote = cross_validate(estimator=vote, X= X_train, y=y_train,
                         return_train_score=True, n_jobs=-1)

In [ ]:
cv_vote

In [ ]:
vote_hong_pred = vote.predict(X_hong)
confusion_matrix(y_hong, vote_hong_pred)

## XXX Random Forest

In [ ]:
forest = RandomForestClassifier(max_depth=7, n_jobs=-1)

In [ ]:
forest.fit(X_train, y_train)

In [ ]:
cv_forest = cross_validate(estimator=forest, X=X_train, y=y_train,
                    return_train_score=True, n_jobs=-1, cv=7)

In [ ]:
cv_forest

In [ ]:
forest_hong_pred = forest.predict(X_hong)
confusion_matrix(y_hong, forest_hong_pred)

In [ ]:
forest_hong_pred

### Randomizedcv

In [ ]:
params = {'n_estimators': scipy.stats.randint(100, 151),
          'max_depth': scipy.stats.randint(3, 8),
          'min_samples_split': scipy.stats.randint(2, 50),
          'min_samples_leaf': scipy.stats.randint(2, 50),
          'min_impurity_decrease': scipy.stats.uniform(0.0001, 0.001)}

In [ ]:
rand_cv = RandomizedSearchCV(estimator=forest,
                             param_distributions=params,    # 파라미터들의 확률 분포
                             n_iter=100,                    # 파라미터들의 조합 갯수
                             n_jobs=-1)

In [ ]:
rand_cv.fit(X, y)

In [ ]:
rand_cv.best_params_

In [ ]:
rand_cv.best_score_

In [ ]:
best_forest = rand_cv.best_estimator_

In [ ]:
bestfor_pre = best_forest.predict(X_hong)

In [ ]:
confusion_matrix(y_hong, bestfor_pre)

## XXX Bagging

In [ ]:
bagging = BaggingClassifier(estimator=DecisionTreeClassifier(),
                            n_estimators=50,
                            oob_score=True, # out of bagging -> 훈련에 한 번도 뽑히지 않은 세트 -> 검증 세트로 사용.
                            n_jobs=-1)

In [ ]:
bagging.fit(X_train, y_train)

In [ ]:
cv_bagging = cross_validate(estimator=bagging, X=X_train, y=y_train,
                    return_train_score=True, n_jobs=-1, cv=10)

In [ ]:
cv_bagging

## OOO GradientBoosting

In [ ]:
gb_clf = GradientBoostingClassifier()

In [ ]:
gb_clf.fit(X, y)

In [ ]:
gb_clf.score(X_hong, y_hong)

In [ ]:
gb_pred_hong = gb_clf.predict(X_hong)

In [ ]:
confusion_matrix(y_hong, gb_pred_hong)

### Randomizedcv

In [ ]:
params = {'max_depth': scipy.stats.randint(3, 8),
          'n_estimators': scipy.stats.randint(50, 151),
          'min_samples_split': scipy.stats.randint(2, 50),
          'min_samples_leaf': scipy.stats.randint(2, 50),
          'min_impurity_decrease': scipy.stats.uniform(0.0001, 0.001)}

In [ ]:
rand_cv = RandomizedSearchCV(estimator=gb_clf,
                             param_distributions=params,    # 파라미터들의 확률 분포
                             n_iter=1000,                   # 파라미터들의 조합 갯수
                             n_jobs=-1)

In [ ]:
rand_cv.fit(X, y)

In [ ]:
rand_cv.best_params_

In [ ]:
rand_cv.best_score_

In [ ]:
best_gb = rand_cv.best_estimator_

In [ ]:
best_gb_pre = best_gb.predict(X)
confusion_matrix(y, best_gb_pre)

In [ ]:
best_gb_hong_pre = best_gb.predict(X_hong)
confusion_matrix(y_hong, best_gb_hong_pre)

### 예측값들 DataFrame 생성

In [ ]:
hong_pred_df = pd.DataFrame()

In [ ]:
hongchun_grid

In [ ]:
hong_pred_df['grid_id'] = hongchun_grid

In [ ]:
best_gb_hong_pre

In [ ]:
hong_pred_df['pred_012'] = best_gb_hong_pre

In [ ]:
best_gb_hong_pre == y_hong

In [ ]:
hong_pred_df['pred_TF'] = best_gb_hong_pre == y_hong

In [ ]:
hong_pred_df.to_excel('/content/drive/MyDrive/중간 프로젝트/Colaboratories/hong_pred_df.xlsx')

In [ ]:
hong_pred_df[(hong_pred_df['pred_012'] == 2) & (hong_pred_df['pred_TF'] == True)]

In [ ]:
hong_pred_df[(hong_pred_df['pred_012'] == 2) & (hong_pred_df['pred_TF'] == False)]

# GradientBoosting -> MinMax PrePro

In [ ]:
no_hong_df = no_hong_df[['wildlife_corridor', 'road_count', 'speed_limit', 'white_road_grade', 'pig_count', 'rani_count', 'dist_to_eco_m']]
yes_hong_df = yes_hong_df[['wildlife_corridor', 'road_count', 'speed_limit', 'white_road_grade', 'pig_count', 'rani_count', 'dist_to_eco_m']]

In [ ]:
mm = MinMaxScaler()

no_hong_df_mm = pd.DataFrame(mm.fit_transform(no_hong_df), columns=no_hong_df.columns)
yes_hong_df_mm = pd.DataFrame(mm.fit_transform(yes_hong_df), columns=yes_hong_df.columns)

In [ ]:
X_no_hong_df_mm = no_hong_df_mm[['wildlife_corridor', 'road_count', 'speed_limit', 'white_road_grade', 'pig_count', 'rani_count', 'dist_to_eco_m']].values
X_yes_hong_df_mm = yes_hong_df_mm[['wildlife_corridor', 'road_count', 'speed_limit', 'white_road_grade', 'pig_count', 'rani_count', 'dist_to_eco_m']].values

In [ ]:
gb_clf2 = GradientBoostingClassifier()

In [ ]:
params = {'max_depth': scipy.stats.randint(3, 8),
          'n_estimators': scipy.stats.randint(50, 151),
          'min_samples_split': scipy.stats.randint(2, 50),
          'min_samples_leaf': scipy.stats.randint(2, 50),
          'min_impurity_decrease': scipy.stats.uniform(0.0001, 0.001)}

rand_cv2 = RandomizedSearchCV(estimator=gb_clf2,
                             param_distributions=params,    # 파라미터들의 확률 분포
                             n_iter=1000,                   # 파라미터들의 조합 갯수
                             n_jobs=-1)

In [ ]:
rand_cv2.fit(X_no_hong_df_mm, y)

In [ ]:
best_gb2 = rand_cv2.best_estimator_

In [ ]:
best_gb_pre2 = best_gb2.predict(X_no_hong_df_mm)
confusion_matrix(y, best_gb_pre2)

In [ ]:
best_gb_hong_pre = best_gb.predict(X_hong)
confusion_matrix(y_hong, best_gb_hong_pre)

In [ ]:
hong_mm_pred_df = pd.DataFrame()

In [ ]:
hong_mm_pred_df['grid_id'] = hongchun_grid

In [ ]:
hong_mm_pred_df['pred_012'] = best_gb_hong_pre

In [ ]:
hong_mm_pred_df['pred_TF'] = best_gb_hong_pre == y_hong

In [ ]:
hong_mm_pred_df[(hong_mm_pred_df['pred_012'] == 2) & (hong_mm_pred_df['pred_TF'] == True)]

In [ ]:
hong_mm_pred_df[(hong_mm_pred_df['pred_012'] == 2) & (hong_mm_pred_df['pred_TF'] == False)]

# <정리>

~~~
모델링은 RandomForest 사용.

찐 테스트세트의 X로 예측한 값 중 2로 나온 값이 우리가 생태 통로를 지어야할 곳.
-> 2로 나온 데이터가 어떤 그리드인지 파악.

만약 2로 나온 그리드가 생태 통로가 많다면,
(1) 생태 통로를 보수
(2) 생태 통로를 추가 설치

찐찐 테스트세트(yes 홍천)의 y 값은 오픈 x.

gridsearch로 최적의 파라미터 찾기.
~~~

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/중간 프로젝트/코랩으로 옮길 엑셀들/DF_final_final.xlsx')

In [ ]:
hongchun = df[df['grid_id'].isin(hongchun_grid)]

In [ ]:
sum(hongchun['total_rk'])

48

In [ ]:
no_hongchun = df[~df['grid_id'].isin(hongchun_grid)]

In [ ]:
sum(no_hongchun['total_rk'])

378